# Aim

In this itteration of the analysis, I want to:
 1. Remove partial sequences even if they are azolla/salivinia or guide sequence
 2. add stronger outgroup
 
Then, re-allign, trim and infer tree as done before.

Point 1. has been taken care of in a file `data/MIKC_orthogroup_selection-basal-v8_guide-v4.list`. 
This file was made by opening `MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5.fasta` in seaview, then deleting all sequences that have too big portions of sequence missing.
When this was done before, exceptions were made for sequences of _Azolla_ and _Salvinia_ and _Arabidopsis_. 
This time, no exceptions.
I wonder whether this will improve the signal for the major clades.

Second, I will add a couple of algal sequences that toghether constitute an outgroup.
I wonder if this might help the signal for those nodes closest to the outgroup.

I'll add the algeal sequences listed below back into the alignment as well.
 - tr_Q5KU24_Q5KU24_COLSC_MADS-box_transcription_factor_CsMADS1_OS_Coleochaete_scutata_OX_3125_GN_csmads1_PE_2_SV_1
 - MNNM-2042156-Cosmarium_granatum
 - TPHT-2126301-Spirotaenia_sp.
 
Add LAMB MADSbox genes for annotation purposes.


In [25]:
grep -A 1 -f <(echo -e "Q5KU24\nMNNM-2042156-Cosmarium_granatum\nTPHT-2126301-Spirotaenia_sp.") \
        ./data/MIKC_orthogroup_selection-basal-v6_guide-v4.fasta \
      > ./data/MIKC_orthogroup_selection-basal-v8_guide-v4.fasta

(phylogenetics) 

: 1

In [26]:
grep -A1 -f ./data/MIKC_orthogroup_selection-basal-v8_guide-v4.list \
            ./data/MIKC_orthogroup_selection-basal-v6_guide-v4.fasta \
         >> ./data/MIKC_orthogroup_selection-basal-v8_guide-v4.fasta

(phylogenetics) 

: 1

In [27]:
cat data/LAMB_X.fasta >> ./data/MIKC_orthogroup_selection-basal-v8_guide-v4.fasta

(phylogenetics) 

: 1

In [28]:
grep '>' -c ./data/MIKC_orthogroup_selection-basal-v*_guide-v4.fasta

./data/MIKC_orthogroup_selection-basal-v3_guide-v4.fasta:425
./data/MIKC_orthogroup_selection-basal-v4_guide-v4.fasta:235
./data/MIKC_orthogroup_selection-basal-v5_guide-v4.fasta:259
./data/MIKC_orthogroup_selection-basal-v6_guide-v4.fasta:188
./data/MIKC_orthogroup_selection-basal-v7_guide-v4.fasta:165
./data/MIKC_orthogroup_selection-basal-v8_guide-v4.fasta:135
(phylogenetics) 

: 1

In [29]:
grep '>' ./data/MIKC_orthogroup_selection-basal-v8_guide-v4.fasta | head

>tr|Q5KU24|Q5KU24_COLSC MADS-box transcription factor CsMADS1 OS=Coleochaete scutata OX=3125 GN=csmads1 PE=2 SV=1
>MNNM-2042156-Cosmarium_granatum
>TPHT-2126301-Spirotaenia_sp.
>cgMADS1_BAD88436
>CRM1_CAA69276
>CRM2_CAA69277
>CRM3_CAA69407
>CRM4_CAA69408
>CRM5_CAA69409
>CRM6_CAA69410
(phylogenetics) 

: 1

# align & trim

In [1]:
inseq='MIKC_orthogroup_selection-basal-v8_guide-v4'

In [31]:
rm "./data/alignments_raw/$inseq"_aligned-mafft-einsi.fasta
conda activate phylogenetics
if    [ ! -d ./data/alignments_raw/ ]
then  mkdir  ./data/alignments_raw
fi
if    [ ! -f "./data/alignments_raw/$inseq"_aligned-mafft-einsi.fasta ]
then  einsi --thread $(nproc) data/$inseq.fasta > ./data/alignments_raw/"$inseq"_aligned-mafft-einsi.fasta
fi
conda deactivate

(phylogenetics) (phylogenetics) (phylogenetics) outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
All-to-all alignment.
tbfast-pair (aa) Version 7.407
alg=N, model=BLOSUM62, 2.00, -0.00, -0.00, noshift, amax=0.0
4 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  130 / 135
done.

Progressive alignment ... 
STEP   114 /134 (thread    3) 
Reallocating (by thread 0) ..done. *alloclen = 1948
STEP   134 /134 (thread    1) 
done.
tbfast (aa) Version 7.407
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
4 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 4
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.

  130 / 135
Segment   1/  1    1- 831
005-0266-1 (thread    3) worse         
Converged2.
done
dvtditr (aa) Version 7.407
alg=A, model

In [33]:
rm ./data/alignments_trimmed/$inseq*

In [4]:
conda activate phylogenetics
if    [ ! -d data/alignments_trimmed ]
then  mkdir  data/alignments_trimmed 
fi

#gt=( 1 2 3 4 5 6 7 8 )
gt=1
# define appendix only once here:
for g in ${gt[@]}
do  trimappendix=trim-auto
    for a in "data/alignments_raw/$inseq"_aligned*.fasta
    do  appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
        if    [ ! -f "$appendix"_"$trimappendix".fasta ]
        then  echo "trimming alignment $a with gap threshold $g"
              sed -i 's/ /_/g' $a
              trimal -in $a   \
                     -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta \
                     -automated1 \
                     -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html &
        fi
    done
done
wait
conda deactivate

(phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi.fasta with gap threshold 1
[1] 32268
(phylogenetics) [1]+  Done                    trimal -in $a -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta -automated1 -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html
(phylogenetics) 

In [35]:
rm data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt2-Mdomain.*
conda activate phylogenetics
trimal -in  data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt2.fasta \
       -selectcols { 61-240 } \
       -out data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt2-Mdomain.fasta
conda deactivate

(phylogenetics) (phylogenetics) 

In [36]:
conda activate jalview
for   i in data/alignments_trimmed/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null &
      fi
done
wait
conda deactivate

(jalview) [1] 4394
(jalview) [1]+  Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
(jalview) 

The Final alignment looks like this, quite happy with it. 
### gt 1
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt1.png)

### gt 2
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt2.png)

### gt 3
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt3.png)

### gt 4
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt4.png)

### gt 5
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt5.png)

### gt 6
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt6.png)

### gt 7
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt7.png)

### gt 8
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt8.png)

### auto
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-auto.png)

### Mdomain
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt2-Mdomain.png)


Now there is new sequences that look too gappy, but that's how things go. Let's move on

# Tree inference

In [5]:
tail $iqprefix.stderr

Jackknife proportion must be between 0.0 and 1.0


In [6]:
conda activate phylogenetics
for a in data/alignments_trimmed/"$inseq"_aligned*auto.fasta
do  iqpendix='iqtree-j30'
    #iqpendix='iqtree-bb2000-alrt2000'

    file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    
    if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
    then echo "Making a directory $file_appendix to store trees (name based on alignment filename)"
         mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
    fi

    iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
    if   [ ! -f "$iqprefix".treefile ]
    then echo "making a tree of file $a"
         nice iqtree -s $a \
                     -m 'JTT+R7' \
                     -j 0.3  \
                     -nt AUTO \
                     -ntmax $(nproc)  \
                     -pre  "$iqprefix" \
                     2>   "$iqprefix".stderr \
                      >    "$iqprefix".stdout
    cat "$iqprefix".log | mail -s "IQtree_run jackknife $a" laura
    fi
done
conda deactivate

(phylogenetics) Making a directory aligned-mafft-einsi_trim-auto to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-auto.fasta
(phylogenetics) 

In [7]:
conda activate phylogenetics
    for a in data/alignments_trimmed/"$inseq"_aligned*Mdomain.fasta
do  iqpendix='iqtree-j30'
    #iqpendix='iqtree-bb2000-alrt2000'

    file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    
    if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
    then echo "Making a directory $file_appendix to store trees (name based on alignment filename)"
         mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
    fi

    iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
    if   [ ! -f "$iqprefix".treefile ]
    then echo "making a tree of file $a"
         nice iqtree -s $a \
                     -m 'JTT+R7' \
                     -j 0.3  \
                     -nt AUTO \
                     -ntmax $(nproc)  \
                     -pre  "$iqprefix" \
                     2>   "$iqprefix".stderr \
                      >    "$iqprefix".stdout
    cat "$iqprefix".log | mail -s "IQtree_run jackknife $a" laura
    fi
done
conda deactivate

(phylogenetics) Making a directory aligned-mafft-einsi_trim-gt2-Mdomain to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt2-Mdomain.fasta
(phylogenetics) 

# observations

Looking at just the alignments, I like -gt `.5` the most. 
However looking at the trees, I'm continously surprised by how much some clades jump around. 
That is for different trims of this version, but also in between versoins.
The algal outgroup does not cluster with lycophytes and bryophytes as I have observed before, but instead with a fern clade or in the middle of a seed-plant clade. 
Then it does cluster close to lycophytes and bryophtes again in `-gt .3`
The homeotic B genes are well separated from their ACDE counter parts, which is an improvement over tree basalv6 for example.
I should draw this tree unrooted, that is sure. (Except for the `-gt .3`, that looks ok)

Prioritising a propper speciation pattern over if I like the alignment or not, I'm inclined to go for tree `-gt .3`.
Our main observation, that our fern clade of interest is a sister clade of homeotic ACDE genes, stands in practically all trees I made and also in this particular trim of version basal8.
Homeotic B genes seem to be older, which also makes sense with literature.
Finally, lycophytes and bryophtes are drawn close to algae which corresponds to trees published before and the expected speciation pattern.
Ideally, I would make a normal bootstrap version of this tree but I doubt very much I get the time to wait a couple of days for that tree to finish.

Actually, I can draw a root in this particular trim, which I think is required for adding extra layers of data. 




On second thought, I can make the 'normal bootstrap' with just ~~100~~200 bootstraps in stead of 1000 to save some time.

In [1]:
grep Best-fit ./analyses/MIKC_orthogroup_selection-basal-v8_guide-v4_trees/*/*log

./analyses/MIKC_orthogroup_selection-basal-v8_guide-v4_trees/aligned-mafft-einsi_trim-gt1/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt1_iqtree-bb2000-alrt2000.log:Best-fit model: JTT+R7 chosen according to BIC
./analyses/MIKC_orthogroup_selection-basal-v8_guide-v4_trees/aligned-mafft-einsi_trim-gt2/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt2_iqtree-bb2000-alrt2000.log:Best-fit model: JTT+R7 chosen according to BIC
./analyses/MIKC_orthogroup_selection-basal-v8_guide-v4_trees/aligned-mafft-einsi_trim-gt3/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt3_iqtree-bb2000-alrt2000.log:Best-fit model: JTT+R7 chosen according to BIC
./analyses/MIKC_orthogroup_selection-basal-v8_guide-v4_trees/aligned-mafft-einsi_trim-gt4/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt4_iqtree-bb2000-alrt2000.log:Best-fit model: JTT+R6 chosen according to BIC
./analyses/MIKC_orthogroup_selection-basal-v8_guide-

The 200 non parametric bootstrap tree confirms my suspisions that support for the upper, more interesting nodes is crap. 
I'm running out of ideas, and definetely out of time to improve upon this, if that is even possible.
Perhaps the information is simply not in the amino acid alignment.

PI want's to submit paper today, but I'll continue improving version8 of the tree after wards, just to be sure.
One thing I messed up is that too much Azolla sequences are missing, this I will fix later. 
First version7 must be pretefied and annotated into publication figure format.

The nonparametric tree, missing some Azfi sequences of interest looks like this:
![nonparametric tree missing azfi seqs](analyses/MIKC_orthogroup_selection-basal-v8_guide-v4_trees/aligned-mafft-einsi_trim-gt5/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt5_iqtree-b200_missingAzfiseqs.png)

Second, I realised I shouldn't compare trees as much by their bootstrap values on nodes I find important, but also on overal likelyhood and the BIC AIC, etcetera.
So let's do that for the shape of it.

In [12]:
for a in ./analyses/MIKC_orthogroup_selection-basal-v8_guide-v4_trees/*/*.iqtree
do  echo $a | cut -d '/' -f 4-
    grep 'Log-likelihood of the tree' -A 5 $a
    echo -e '\n'
done

aligned-mafft-einsi_trim-gt1/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt1_iqtree-bb2000-alrt2000.iqtree
Log-likelihood of the tree: -32950.7081 (s.e. 1183.3016)
Unconstrained log-likelihood (without tree): -1704.4326
Number of free parameters (#branches + #model parameters): 279
Akaike information criterion (AIC) score: 66459.4162
Corrected Akaike information criterion (AICc) score: 74682.5741
Bayesian information criterion (BIC) score: 67491.8399


aligned-mafft-einsi_trim-gt2/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi_trim-gt2_iqtree-bb2000-alrt2000.iqtree
Log-likelihood of the tree: -30663.8011 (s.e. 1015.1381)
Unconstrained log-likelihood (without tree): -1321.8361
Number of free parameters (#branches + #model parameters): 279
Akaike information criterion (AIC) score: 61885.6021
Corrected Akaike information criterion (AICc) score: 218125.6021
Bayesian information criterion (BIC) score: 62857.8604


aligned-mafft-einsi_trim-gt3/MIKC_o

So less information put in, is a higher likelyhood because less noise... 
Perhaps this is only usefull when comparing trees made with different models.

In [13]:
for a in ./analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/*/*.iqtree
do  echo $a | cut -d '/' -f 4-
    grep 'Log-likelihood of the tree' -A 5 $a
    echo -e '\n'
done

aligned-mafft-einsi_trim-gt1/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt1_iqtree-bb2000-alrt2000.iqtree
Log-likelihood of the tree: -43245.2804 (s.e. 1494.8994)
Unconstrained log-likelihood (without tree): -2187.9961
Number of free parameters (#branches + #model parameters): 337
Akaike information criterion (AIC) score: 87164.5607
Corrected Akaike information criterion (AICc) score: 94283.6857
Bayesian information criterion (BIC) score: 88483.4112


aligned-mafft-einsi_trim-gt2/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt2_iqtree-bb2000-alrt2000.iqtree
Log-likelihood of the tree: -37337.3596 (s.e. 1196.7915)
Unconstrained log-likelihood (without tree): -1393.4161
Number of free parameters (#branches + #model parameters): 341
Akaike information criterion (AIC) score: 75356.7191
Corrected Akaike information criterion (AICc) score: 308600.7191
Bayesian information criterion (BIC) score: 76560.2544


aligned-mafft-einsi_trim-gt3/MIKC_o

So that basalv8 trees have heigher likelyhood, but can I really compare these vallues in between different trees based on different alignments. 
Let's dive into literature again.

# obersvations
Now the v7 nonparametric tree finsihed, I find that boostrap support is very low even at the base of the important clades.
I doubt whether the minimal changes I made in v8 will substantially improve that support.
Instead I'm now doing a quick test with partitionfinder 2 on the basalv8 gt5 alignment, feeding it the blocks for the M I K and C domains separatelly. 

Partitionfinder found it's best fitting to have the MIKC domains as one partition with model LG+I+G.

# Improvement atempts 

## Alternatives for support assemessment
Going over my phylogeny notes, I read that traditional bootstrap can perform quite bad with sequences that are very similar too each other.
It seems that's my case here.

Bayesian prosterior probabilities should perform better for data with such low divergence. 
I did one tree with BEAST before, but I really should make multiple and gain some experience.
Addtionally, I don't quite know how to choose my priors in an informed way.

Alternativelly, jackkniffing should be more robust.
Although that's hardly done anymore, I did notice it's implemented in IQtree. 
I might give it a try since it's just a simple parameter change in the code I already have.

Finally, I had this paper about transfer bootstraps (opposed to the normal Felsensteins bootstrap). 
This might also deal better with lowly divergent data as is the case for me. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6030568/pdf/nihms947517.pdf
I'll try to install this and have a go at the bootstrap trees I already created before.

# Shorter alignments.
Perhaps there is too much ambiguity in the alignment, hence I'm making a stricter trim of the alignment.
One using the auto setting in trimAL, which I have found to be very strict before and is said to be optimised for ML trees.
A second containing only the M domain. 
Of both I'll make phylogenies again and then conclude.

The auto alignment is trimmed with the trimal loop above, the M domain in a separate code cell. Now let's make both trees.

# Improvement conclusions


### short alignments & Jackknifing
Looking at the trees I made, 4 in total: one with only the `M domain`, or the `trimal --automated1` version. 
Both were run with non-parametric b200, and with jackknife.
The jackknife versions don't have support vallues at the nodes, and after reading more about jackknife I don't like it that much. 
The alignment is so short already, to make bootstrap trees with even less information, doesn't seem to make sense to me.
Then, talking about the nonparametric trees infered from both these short alignments. 
Long story short, these are crap too (excuse my language).
The clades are polyphyletic and support is a disaster.

### Transfer bootstraps
I also took the time to read the transferbootstrap paper.
After reading I'm quite convinced this is a nice metric, although I dislike adding another metric to the already confusing sea of support vallues like SH-aLRT, UFbootstrap and nonparametric bootstrap.
But I realise that support vallue is not a statistical test or hypothesis testing method, it's just a number indicating the % of times a certain clade was retrieved in identical fasion in a bootstrap tree.
Rather than requiring this clade is identical to count for the bootstrap value (binary), the authors propse to use a distance franction.
Hence the support vallue can be read as the degree of similarity comparing a certain clade with the bootstrap trees.
This metric is a lot more informative for bigger trees, and trees were some clades tend to move around a lot.
Provided of course that the interpretation is explained in the 

open question still, the paper said something about taxon stability, let's see if I can calculate and visualise that as well.


### Baysian trees
Although I would actually like to get better at running beast for bayesian trees, I just don't have the time to invest for "just a supplementary figure". Perhaps another time.

# Basalv9
The dataset can be slightly optimised still: removing rogue taxa and very long branches. 
- AGL33
- tr_Q9FYZ8_Q9FYZ8_SPIAN_MADS-box_protein_LAMB1_OS_Spinulum_annotinum_OX_13840_GN_MB1_PE_2_SV_1
- ENQF-2004631-Lycopodium_annotinum
- ENQF-2004632-Lycopodium_annotinum
- TPHT-2126301-Spirotaenia_sp.

Although in doubt, I'm inclined to remove also
- CVEG-2123825-Azolla_cf._caroliniana
- PIVW-2019502-Ceratopteris_thalictroides
- SmMADS6

The former two always end up together. 
I find it very suspicious I don't have an Azolla filiculoides version for this gene, and since it also ends up with SmMADS6 (which jumps around a lot), I think this is a fluke.

The algal outgroup is oddly placed, should I add more algal sequences to see if it moves out of a seedplant clade.
It has been monophyletic in some trees, at least nearly all algeal sequences were in a single clade in basalv6. 
I may add the basalv6 algal outgroup again in v9; I should check how much it really differs.
By the way, looking back at basalv6, that was actually quite decent except for the extremely short branch lengths separating the major clades.
Those miniscule branches with practically absent support were the reason I moved on the v7 and v8 in the first place.
Let's for the fun of it, look at basalv6 using the transfer bootstrap, and I'll try to assess taxon instability as well.

![basalv6 with transfer boostraps and taxon instability](analyses/MIKC_orthogroup_selection-basal-v6_guide-v4_trees/aligned-mafft-einsi_trim-gt1-seqrmmanual/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual_iqtree-MFP-b1000_booster.png)








Interestingly, this indicates more unstable / rogue taxa than I had identified by eye. 
Those I had identified by eye score high on this metric shown here, but other taxa scoring equally as bad, weren't picked up by me and I wonder if they should...

For example, all algal sequences are considered instable. 
Perhaps these are prone to shuffling within the clade which I didn't pick up on since I coloured them as a single clade.
The two lycophyte sequences on the extrememly long branch score exceptionally high and support my plan to remove these from the phylogeny.
In concordance with high bootstrap values, to older fern clade is very stable.
Then comes a big mess of seedplant taxa that are all considered instable.
Interestingly enough, the one SmMADS6 sequence that alsways seems to find itself in odd places doesn't score very high on onstability.

Within this big clade, and actually throughout the entire tree, Salvinia sequences are indicated to be unstable.
This is in cordance with previous observations on Salvina sequences moving a lot, having very long branches, or landing in the middle of a seedplant clade.
The annotation of Salvinia leaves room for improvent still.

Being pragmatic, I'll judge the instability of a taxon by comparing it's value to that of other taxa in it's vacinity in the tree.

### diff v6 vs v8
Looking at v8 again, I'm wondering if I haven't removed too much... Let's see

In [2]:
diff <(grep '>' data/alignments_raw/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi.fasta | tr -d '>' | sort) \
     <(grep '>' data/alignments_raw/MIKC_orthogroup_selection-basal-v8_guide-v4_aligned-mafft-einsi.fasta | tr -d '>' | sort) 

16,26d15
< Azfi_s0001.g000595_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0003.g007710_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0003.g007732_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0009.g011722_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0009.g011725_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0009.g011735_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0009.g011751_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0009.g011788_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0009.g011792_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0016.g014286_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s0016.g014288_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
28,35d16
< Azfi_s0028.g024032_sequence_match_in_blast_db_Azolla_filiculoides_protein_v1.1
< Azfi_s00

: 1

That's definitely more than I'd like to have removed... Let's look at basalv7 instead
### basalv6 vs v7

In [4]:
diff <(grep '>' data/alignments_raw/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi.fasta | tr -d '>' | sort) \
     <(grep '>' data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta | tr -d '>' | sort) 

41d40
< Chlre_v5_5-Cre11_g467577_t1_1-Chlamydomonas_reinhardtii
66d64
< FQLP-2004170-Klebsormidium_subtile
68,76d65
< JVSZ-2004872-Equisetum_hymale
< JVSZ-2008957-Equisetum_hymale
< JVSZ-2012388-Equisetum_hymale
< JVSZ-2014398-Equisetum_hymale
< JVSZ-2014659-Equisetum_hymale
< JVSZ-2014698-Equisetum_hymale
< JVSZ-2127731-Equisetum_hymale
< JVSZ-2127794-Equisetum_hymale
< JVSZ-2127811-Equisetum_hymale
91,95d79
< MA_130755g0010
< MA_13933g0010
< MA_17919g0010
< MA_6805g0010
< MNNM-2042156-Cosmarium_granatum
137d120
< Pp014G086100
145d127
< QPDY-2028842-Coleochaete_irregularis
169,172d150
< TPHT-2010795-Spirotaenia_sp.
< TPHT-2126301-Spirotaenia_sp.
< tr|Q5KTX4|Q5KTX4_9VIRI_MADS-box_protein_CpMADS1_(Fragment)_OS=Closterium_peracerosum-strigosum-littorale_complex_OX=34146_GN=CpMADS1_PE=2_SV=1
< tr|Q5KU24|Q5KU24_COLSC_MADS-box_transcription_factor_CsMADS1_OS=Coleochaete_scutata_OX=3125_GN=csmads1_PE=2_SV=1
184d161
< XMGP-2008586-Juniperus_scopulorum


: 1

# conclusion

So basal v9 shall be based on basalv7. 
Wierd stuff happend in basalv8.
I'll try to have again the solid outgroup of v6, but the sleightly better separation between clades as in basalv8, although one can argue if it's really so much of an improvement.

I'll swap some taxa in and out, create the einsi alignment optimising for retention of columns. 
Then, I might align with prank after MAFFT einsi, then compare alignments and resulting trees. 
I found my notes on prank again and decided I would give it a try, even though it crashed on my countless times before.

Finally I'll run the non-parametric iqtree and create transfer bootstraps with booster, while including taxon instability. (Make sure I have a good and short explanation of how to interpret that for a figure legend).